In [11]:
import pandas as pd
import os
from tqdm.notebook import tqdm

# Define the folder path
folder_path = './newsdata'

# Initialize an empty list to store dataframes
dataframes = []

# Iterate through all files in the folder
for file in tqdm(os.listdir(folder_path)):
    if file.endswith('.xlsx'):
        # Read each Excel file into a dataframe
        file_path = os.path.join(folder_path, file)
        df = pd.read_excel(file_path, engine="openpyxl")
        df['date'] = file[-13:-5]
        dataframes.append(df)

# Concatenate all dataframes into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame to the user
#import ace_tools as tools; tools.display_dataframe_to_user(name="Combined News Data", dataframe=combined_df)


  0%|          | 0/366 [00:00<?, ?it/s]

In [13]:
df = combined_df[['date','title','information']]

In [15]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

/var/folders/wb/hsgyhgm57kbgysv0zmm8fjgc0000gn/T/ipykernel_54145/292591770.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')


In [ ]:
df.sort_values('date', inplace=True)
df.reset_index(drop=True, inplace=True)

/var/folders/wb/hsgyhgm57kbgysv0zmm8fjgc0000gn/T/ipykernel_54145/2566996094.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values('date', inplace=True)


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146333 entries, 0 to 146332
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         146333 non-null  datetime64[ns]
 1   title        146333 non-null  object        
 2   information  145629 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 3.3+ MB


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import networkx as nx
import matplotlib.pyplot as plt

/var/folders/wb/hsgyhgm57kbgysv0zmm8fjgc0000gn/T/ipykernel_54145/1534405278.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=["information"], inplace=True)  # 결측치 제거
/var/folders/wb/hsgyhgm57kbgysv0zmm8fjgc0000gn/T/ipykernel_54145/1534405278.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['information'] = df['information'].astype(str).str.strip()  # 문자열로 변환 및 공백 제거
/var/folders/wb/hsgyhgm57kbgysv0zmm8fjgc0000gn/T/ipykernel_54145/1534405278.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

TypeError: operands could not be broadcast together with shapes (20,) (17,) 

In [33]:
# 1. 데이터 전처리
df.dropna(subset=["information"], inplace=True)  # 결측치 제거
df['information'] = df['information'].astype(str).str.strip()  # 문자열로 변환 및 공백 제거
df['month'] = df['date'].dt.to_period('M')  # 월별 그룹화

df = df[df['information'].str.len() > 10]  # 텍스트 길이가 10자 이상인 경우만 남김

In [35]:
# 2. 키워드 추출 함수
def extract_keywords(text):
    tfidf = TfidfVectorizer(max_features=20, stop_words="english")
    try:
        tfidf_matrix = tfidf.fit_transform([text])
        keywords = tfidf.get_feature_names_out()
    except ValueError:
        keywords = []  # 빈 텍스트인 경우 빈 리스트 반환
    return keywords

df['keywords'] = df['information'].apply(extract_keywords)

In [36]:
# 3. 월별 데이터 그룹화 및 키워드 관계 계산
monthly_keyword_networks = {}
for month, group in df.groupby('month'):
    all_keywords = group['keywords'].sum()  # 해당 월의 모든 키워드 모음
    unique_keywords = list(set(all_keywords))
    if not unique_keywords:  # 유효한 키워드가 없는 경우 건너뜀
        continue

    # 키워드 임베딩 생성
    keyword_embeddings = {}
    for kw in unique_keywords:
        try:
            embedding = keyword_extractor(kw)[0][0]  # Transformer 기반 임베딩
            if len(embedding) != 768:  # 임베딩 크기 확인 (예: BERT 기반 모델은 768차원)
                continue
            keyword_embeddings[kw] = embedding
        except Exception as e:
            print(f"Error in embedding {kw}: {e}")
            continue

    if len(keyword_embeddings) < 2:  # 임베딩된 키워드가 2개 미만이면 건너뜀
        continue

    similarities = cosine_similarity(list(keyword_embeddings.values()))

    # 그래프 생성
    G = nx.Graph()
    for i, kw1 in enumerate(keyword_embeddings.keys()):
        for j, kw2 in enumerate(keyword_embeddings.keys()):
            if i != j and similarities[i, j] > 0.5:  # 유사도가 0.5 이상인 키워드만 추가
                G.add_edge(kw1, kw2, weight=similarities[i, j])

    monthly_keyword_networks[month] = G


TypeError: operands could not be broadcast together with shapes (20,) (17,) 

In [ ]:
# 4. 월별 관계망 시각화
for month, G in monthly_keyword_networks.items():
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_size=50, font_size=8)
    plt.title(f"Keyword Network - {month}")
    plt.show()


In [31]:
df

,date,title,information,month,keywords
0,2023-12-01,"“고금리, 은행엔 좋은거 아니야?”…저축은행 순이익 날아갔다는데",저축銀 3분기 공시 발표 고금리에 수익성 악화로 부진경기둔화에 하반기 실적 악화 ...,2023-12,"[3분기, 5개사, 65, 감소했다, 것으로, 고금리, 당기순손실을, 당기순이익은,..."
1,2023-12-01,시중은행 '부실 대응 자본' 대손준비금 아직까진 '역주행',충당금 늘며 필요성 줄었지만고금리 리스크 확산에 우려도금융당국 새 규제 가동 '촉각...,2023-12,"[4대, 금융당국이, 대손준비금, 대손준비금은, 대손준비금이, 대출, 됐다, 된다,..."
2,2023-12-01,"한은 ""美주도 산업정책, 생산·고용 확대…韓에도 '긍정적'""",조 바이든 미국 대통령이 대만 TSMC가 미 애리조나주 피닉스에 짓고 있는 공장건설...,2023-12,"[것으로, 관련, 그간, 긍정적, 나온다, 다만, 미국, 반도체, 받고, 분석이다,..."
3,2023-12-01,3분기 증권사 순이익 14%↓…IB 위축·부동산 평가손실 영향,증권사[연합뉴스TV 캡처] (서울=연합뉴스) 오지은 기자 = 국내 증권사들의 ...,2023-12,"[3분기, ib, 감소, 감소한, 감소했다, 것으로, 관련, 국내, 기준, 대비, ..."
4,2023-12-01,"""내년 경제도 춥다"" 고금리 속 물가전망치 높아져…성장은 2%대 고착 우려","한은, 내년 물가 눈높이 석달새 높여 잡아…2.4→2.6%성장률, 내년도 내후년도 ...",2023-12,"[2년, 경제, 그는, 까지, 내년, 내년도, 당초, 물가, 성장률, 성장률은, 성..."
...,...,...,...,...,...
146328,2024-11-30,[주형연의 에구MONEY] 기준금리 `깜짝 인하`에도 내 대출금리는 요지부동,한국은행 금융통화위원회가 28일 '11월 통화정책방향' 회의에서 기준금리를 두 차례...,2024-11,"[25, 관련된, 금리, 금리가, 금리를, 금융통화위원회가, 금통위가, 기사를, 내..."
146329,2024-11-30,오늘부터 후불 기후동행카드 쓴다…어디 혜택 좋을까[카드팁],"후불 기후동행카드, 30일부터 첫 서비스 개시한달마다 충전하는 번거로움 없애…카드별...",2024-11,"[10, 경우, 기후동행카드, 기후동행카드는, 대중교통, 따라, 마을버스, 무제한으..."
146330,2024-11-30,“엔화는 올라봤자…” 은행 엔화예금 1년 만에 ‘최저’[머니뭐니],5대 시중은행 엔화예금 잔액 1년 만에 ‘최저 수준’통화정책 방향 불확실…환차익 노...,2024-11,"[12월, 5대, 가치, 강달러, 금리, 기준, 기준금리를, 따르면, 만에, 서울,..."
146331,2024-11-30,만기매칭형 ETF로 가격변동위험 헤지를![100세시대연구소의 경제수명 늘리기],남창주 100세시대연구소 연구위원 이미지투데이[서울경제] 경제수명 연장을 위해...,2024-11,"[etf, etf를, etf의, 가격변동위험을, 가격이, 같은, 거의, 만기, 만기..."
